In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42)

# Classifying objects

In the following, we will use the old-school CIFAR-10 dataset that contains low-resolution pictures of objects of 10 categories. This - and other - dataset is available as part of the `torchvision` package, which you should install.

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms, utils
# where to put the data
data_path = './'

# this constructs a CIFAR10 dataset, selects the training part of it, 
# downloads it if necessary, and adds additional transforms that we 
# will need to convert each image to a pytorch tensor AND to provide
# a nice conversion of the RGB images into greyscale
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


This has now downloaded the images if not already done so - you will notice that this is quite a "hefty" dataset already at 170MB. Now let's download our validation set or test set.

In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [6]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

## Part1 Pytorch CNN level 1

In [21]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(F.relu(self.conv1(x)), 2)
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [28]:
import datetime 

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, iter_num):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            # put the data onto the correct pytorch calculation device
            imgs=  imgs.to(device =device)
            labels = labels.to (device = device)
            # put a batch through the model
            outputs = model(imgs)
            # determine the loss
            loss = loss_fn(outputs, labels)
            # zero the gradients and determine backpropagation
            optimizer.zero_grad()
            loss.backward()

           # do one step of optimization
            optimizer.step()
            # keep track of the loss
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Random Seed Iteration Number {} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), iter_num+1, epoch,
                loss_train / len(train_loader)))

In [29]:
device = (torch.device('cuda')if torch.cuda.is_available() 
    else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [25]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

# get the model and put it onto the proper device
model = Net().to(device=device)
# standard optimizer 
optimizer = optim.Adam(model.parameters(), lr=1e-4) 
# classification loss
loss_fn = nn.CrossEntropyLoss()

# go for 100 epochs
training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,1
)

2022-11-20 20:31:33.816200 Epoch 1, Training loss 0.6273969275176905
2022-11-20 20:32:30.718738 Epoch 10, Training loss 0.3450257361030123
2022-11-20 20:33:40.708561 Epoch 20, Training loss 0.31299255674432036
2022-11-20 20:34:40.575392 Epoch 30, Training loss 0.288867423868483
2022-11-20 20:35:30.627801 Epoch 40, Training loss 0.2712316007181338
2022-11-20 20:36:26.050142 Epoch 50, Training loss 0.253683311686774
2022-11-20 20:37:16.759181 Epoch 60, Training loss 0.24485198536496253
2022-11-20 20:38:07.569155 Epoch 70, Training loss 0.23441997834831285
2022-11-20 20:38:58.527047 Epoch 80, Training loss 0.2247259388588796
2022-11-20 20:39:49.013212 Epoch 90, Training loss 0.22042216403279336
2022-11-20 20:40:39.372445 Epoch 100, Training loss 0.2126995881746529


In [30]:
torch.save(model.state_dict(), data_path + 'birds_vs_airplanes.pt')

Define 20 different seeds and train this network for 100 epochs with a batch size of
64, ADAM optimizer (learning rate = 1e-4), tracking training and validation accuracy
in EACH epoch.


In [32]:
import random

In [ ]:
models_collection = []
for i in range(20):
    
    seed_int =random.randint(0, 1000) # creating random seed values for the  different datasets
    torch.manual_seed(seed_int)
    train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

    # get the model and put it onto the proper device
    model = Net().to(device=device)
    # standard optimizer 
    optimizer = optim.Adam(model.parameters(), lr=1e-4) 
    # classification loss
    loss_fn = nn.CrossEntropyLoss()

    # go for 100 epochs
    training_loop(n_epochs = 100, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader,iter_num =i )
    

    models_collection.append(model)

2022-11-20 21:35:45.657395 Random Seed Iteration Number 1 Epoch 1, Training loss 0.5906210936558475
2022-11-20 21:36:49.044048 Random Seed Iteration Number 1 Epoch 10, Training loss 0.3262808236536706
2022-11-20 21:37:45.512347 Random Seed Iteration Number 1 Epoch 20, Training loss 0.29588102406946715


In [ ]:
for i, m in enumerate(models_collection):
    torch.save(m.state_dict(), f"test{i}.pth")